In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import rasterio
import random
import shapely
from tqdm import tqdm
import geojson
import rasterio.features
import sys
import cv2
from collections import Counter
import copy

In [2]:
# Open the tif file containing the orthographic map of Denmark 
path = './ortodata_2014_res_2_crop.tif'
dataset = rasterio.open(path)

# Get some statistics about the data, such as the bounds of the coordinates
bounds = dataset.bounds
left, bottom, right, top = np.array(bounds).astype(int)
transform = dataset.transform
crs = dataset.crs
unit = crs.linear_units

print('Bounds:', left, bottom, right, top)
print('Data shape:', dataset.shape)
print('CRS:', crs)

# Extract the dimensions of a pixel from the transform operation
pixel_width = transform[0]
pixel_height = -transform[4]  # The pixel heigh is negative(?)
pixel_area = pixel_width * pixel_height # Calculate the area of a pixel

print(f"Pixel width: {pixel_width} {unit}s")
print(f"Pixel height: {pixel_height} {unit}s")
print(f"Pixel area: {pixel_area} square {unit}s")

Bounds: 440000 6070000 610000 6340000
Data shape: (135000, 85000)
CRS: EPSG:25832
Pixel width: 2.0 metres
Pixel height: 2.0 metres
Pixel area: 4.0 square metres


In [3]:
%%time
# This file contains annotations for different types of nature phenomena (e.g. lakes, forests)
path_to_file = './naturtyper_layer.geojson'

with open(path_to_file, 'r') as f:
    gj = geojson.load(f)
print(len(gj['features']), gj.keys())

# Filter to the annotations of the lakes around Denmark
gj_features = []
for feature in gj['features']:
    if feature['properties']['Natyp_kode'] == 6: # Code for lakes is 6
        gj_features.append(feature)
print(len(gj_features))
gj_features[0]

312325 dict_keys(['type', 'name', 'crs', 'features'])
152920
CPU times: user 1min 12s, sys: 5.56 s, total: 1min 18s
Wall time: 2min 36s


{"geometry": {"coordinates": [[[[501332.248, 6224773.935], [501334.244, 6224779.934], [501334.244, 6224784.933], [501333.246, 6224790.932], [501327.243, 6224789.932], [501317.249, 6224783.933], [501316.243, 6224780.934], [501319.244, 6224774.935], [501324.25, 6224771.936], [501329.247, 6224770.936], [501332.248, 6224773.935]]]], "type": "MultiPolygon"}, "properties": {"Aendr_kode": 0, "Aendrbegr": "Ikke udfyldt", "Besig_dato": null, "Bruger_id": "00000000-0000-0000-0000-000000000000", "CVR_kode": 29189919, "CVR_navn": "Herning kommune", "Gl_sys_ref": null, "Journalnr": null, "Link": null, "Natyp_kode": 6, "Natyp_navn": "Sø", "Objekt_id": "0460cd7c-5353-11e2-af2b-00155d01e765", "Off_kode": 1, "Offentlig": "Synlig for alle", "Oprettet": "2006-12-31T01:00:00", "Oprindelse": "Ikke udfyldt", "Oprindkode": 0, "Sagsbeh": null, "Shape_area": 252.94599999301087, "Shape_length": 0.0, "Status": "Gældende / Vedtaget", "Statuskode": 3, "Systid_fra": "2006-12-31T01:00:00", "Systid_til": null, "Temak

In [4]:
# Get dimensions and transform of the original raster
out_shape = (dataset.height, dataset.width)

# Empty list to store the geometries (polygons)
geometries = []

failed, skipped = 0, 0 # Track which annotations could not be converted

# Loop over the annotations
for i, feature in tqdm(enumerate(gj_features)):
    geometry = copy.deepcopy(feature['geometry'])
    coords = copy.deepcopy(geometry['coordinates'])
    try:
        poly = shapely.geometry.shape(geometry)
        if not poly.is_valid: # Skip invalid shapes
            skipped += 1
            continue
    
        # Get the coordinates of the lake in the form [(x1, y1), (x2, y2), ...]
        coords_xy = np.array(coords).reshape(-1, 2)
        
        # Split x and y coordinates
        x, y = coords_xy[:, 0], coords_xy[:, 1]

        # Ensure we only keep annotations that are within bounds of our map
        x_min, x_max, y_min, y_max = np.min(x), np.max(x), np.min(y), np.max(y)
        if x_min <= bounds.left or x_max >= bounds.right or y_min <= bounds.bottom or y_max >= bounds.top:
            skipped += 1
            continue
        
        # Transform the polygon's coordinates to indices within the orthographic raster
        y_trans, x_trans = rasterio.transform.rowcol(transform, x, y, op = np.round)

        # Convert to a polygon and add to our list of shapes
        poly = shapely.Polygon(zip(x_trans, y_trans))
        
        # Convert to the GeoJSON format for rasterio
        geometries.append((shapely.geometry.mapping(poly), 1))
        
    except Exception as e:
        if failed == 0:
            print(f"Failed to convert annotation {i}: {e}")
        failed += 1
        continue

print('Creating mask...')

# Rasterize all shapes onto a single map
mask = rasterio.features.rasterize(
    geometries, out_shape = out_shape, fill = 0, default_value = 1, dtype = rasterio.uint8   
)

print(f'Failed to convert {failed} annotations')
print(f'Skipped {skipped} annotations')

118it [00:00, 1085.11it/s]

Failed to convert annotation 7: setting an array element with a sequence. The requested array has an inhomogeneous shape after 2 dimensions. The detected shape was (1, 2) + inhomogeneous part.


152920it [03:12, 792.55it/s] 


Creating mask...
Failed to convert 3760 annotations
Skipped 59104 annotations


In [16]:
import pyproj

source_crs = pyproj.CRS.from_epsg(25832)   
target_crs = pyproj.CRS.from_epsg(4326) 
transformer = pyproj.Transformer.from_crs(source_crs, target_crs, always_xy=True)

In [17]:
skipped = 0
failed = 0

coord_area_lst = []


# Loop over the annotations
for i, feature in tqdm(enumerate(gj_features)):
    geometry = feature['geometry']
    coords = geometry['coordinates']
    try:
        poly = shapely.geometry.shape(geometry)
        if not poly.is_valid: # Skip invalid shapes
            skipped += 1
            continue
    
        # Get the coordinates of the lake in the form [(x1, y1), (x2, y2), ...]
        coords_xy = np.array(coords).reshape(-1, 2)
        
        # Split x and y coordinates
        x, y = coords_xy[:, 0], coords_xy[:, 1]

        # Ensure we only keep annotations that are within bounds of our map
        x_min, x_max, y_min, y_max = np.min(x), np.max(x), np.min(y), np.max(y)
        if x_min <= bounds.left or x_max >= bounds.right or y_min <= bounds.bottom or y_max >= bounds.top:
            skipped += 1
            continue

        # Convert to geographic coordinate system
        gcs_coords = [transformer.transform(long, lat) for long, lat in zip(x, y) if isinstance(long, float) and isinstance(lat, float)]
        
        # Transform the polygon's coordinates to indices within the orthographic raster
        y_trans, x_trans = rasterio.transform.rowcol(transform, x, y, op = np.round)
        x_trans_min, x_trans_max, y_trans_min, y_trans_max = int(np.min(x_trans)), int(np.max(x_trans)), int(np.min(y_trans)), int(np.max(y_trans))
        
        mask_crop = mask[x_trans_min:x_trans_max+1, y_trans_min:y_trans_min+1]
        mask_ones = (mask_crop == 1).sum()
        lake_size = mask_ones * pixel_area
        coord_area_lst.append((gcs_coords, lake_size))
        
    except Exception as e:
        if failed == 0:
            print(f"Failed to convert annotation {i}: {e}")
        failed += 1
        continue
print(skipped)

151it [00:00, 1463.50it/s]

Failed to convert annotation 7: setting an array element with a sequence. The requested array has an inhomogeneous shape after 2 dimensions. The detected shape was (1, 2) + inhomogeneous part.


152920it [01:53, 1349.75it/s]

59104


In [22]:
for c in coord_area_lst:
    if c[1] > 0:
        print(c)
        break

([(9.04361131090122, 56.29501711308642), (9.043676277767863, 56.29531351625007), (9.04354710680122, 56.29537644434132), (9.043563385211003, 56.29551126920588), (9.043418134963277, 56.295601138384605), (9.043256619473647, 56.2956820465294), (9.04249726515313, 56.29575416753378), (9.04248123959532, 56.29584400018818), (9.042723823447249, 56.296041629928325), (9.042643303786214, 56.29623927202021), (9.041981119431817, 56.29648212497252), (9.04172254285153, 56.29650915689501), (9.041722758149417, 56.29670677115018), (9.041545040561768, 56.296724799910045), (9.04131875531293, 56.29667097752096), (9.041350809217027, 56.296383525348176), (9.041463774124948, 56.29623967250267), (9.041980577750566, 56.295987985984404), (9.042109397682273, 56.2956015902453), (9.042755244315863, 56.2951970658922), (9.043336518819196, 56.294954326926565), (9.04361131090122, 56.29501711308642)], 8.0)
